In [1]:
import numpy as np
import pandas as pd
import spacy
from multiprocessing import Pool
import re

In [2]:
train = pd.read_pickle("../input_data/train_es.pkl")

In [3]:
nlp = spacy.load('es')

In [4]:
train.head()

,question,answer,story,story_length
25466,"\nMientras el @entity458, @placeholder y @enti...",@entity170,"Desde que la crisis se hizo evidente, la @enti...",1993
25465,"\nMientras el @placeholder, @entity10 y @entit...",@entity126,"Desde que la crisis se hizo evidente, la @enti...",1993
25467,"\nMientras el @entity356, @entity124 y @placeh...",@entity101,"Desde que la crisis se hizo evidente, la @enti...",1991
13068,Grupos armados de 'ex contras' vuelven a rearm...,@entity68,"""Ojalá no volviera a suceder, pero si aquí hub...",1989
7482,"Un cementerio de @placeholder, símbolo del trá...",@entity91,Sobre una de las estelas se lee @entity240 par...,1989


Error 1 (temporarily fixed)

In [5]:
201 * 16

3216

In [21]:
finds = train['answer'].apply(lambda x: re.search(r'@entity445', x))

In [22]:
finds[~pd.isnull(finds)].apply(lambda x: x.group(0))

24635    @entity445
19380    @entity445
9048     @entity445
7604     @entity445
20111    @entity445
8389     @entity445
27815    @entity445
9003     @entity445
9655     @entity445
7174     @entity445
33609    @entity445
54319    @entity445
11296    @entity445
32654    @entity445
10321    @entity445
47978    @entity445
2340     @entity445
32733    @entity445
33028    @entity445
16749    @entity445
22235    @entity445
6521     @entity445
9077     @entity445
3705     @entity445
845      @entity445
57032    @entity445
27985    @entity445
36775    @entity445
353      @entity445
19936    @entity445
            ...    
3917     @entity445
20886    @entity445
40549    @entity445
29909    @entity445
1312     @entity445
29622    @entity445
39363    @entity445
37333    @entity445
29216    @entity445
32373    @entity445
45948    @entity445
7545     @entity445
34220    @entity445
53485    @entity445
27255    @entity445
12289    @entity445
41282    @entity445
29964    @entity445
33886    @entity445


In [11]:
train['story'][27441]

'El proyecto económico-fiscal de país de Podemos es fundamental, para entender qué modelo de política económica quieren implantar en @entity189, sobre qué premisas y cuáles pueden ser sus consecuencias para los españoles. Aunque lo primero que debemos tener en cuenta es que Podemos quiere y defiende un referéndum de independencia para @entity75 -y posiblemente para el resto de comunidades que lo pidan, como el @entity181 realizarse este referéndum y de salir sí, todo este proyecto de ingeniería económico-fiscal quedaría en agua de borrajas, para empezar porque se perdería el 18% del PIB de @entity189 que actualmente supone @entity75. \nPero seamos positivos y pensemos que con o sin consulta, @entity75 seguirá siendo parte de @entity189. El cuadro macroeconómico que contempla la memoria del documento programático es el siguiente:\nLas cifras presentadas trabajan a propósito con ratios sobre PIB, que permiten dibujar un panorama más difuso y favorable a la propuesta. Traducción: los econ

Error 2:

In [15]:
train.iloc[29521]['question']

'\nLas autoridades de @placeholder han ahogado la protesta estudiantil tras dos meses y medio '

In [10]:
d = nlp(train.iloc[29521]['question'])

In [11]:
d


Las autoridades de @placeholder han ahogado la protesta estudiantil tras dos meses y medio 

In [12]:
[w.has_vector for w in d]

[False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [7]:
# def lengths(x):
#     return len(nlp(x, entity=False, parse=False, tag=False))

# with Pool(4) as p:
#     train['s_len'] = list(p.map(lengths, train['story']))

In [12]:
bn = 461
focus = train.iloc[(bn - 1) * 8: (bn + 20 - 1) * 8, :]

In [17]:
def get_word_ids(doc, rnn_encode=False, max_length=100, nr_unk=100):
    queue = list(doc)
    X = np.zeros(max_length, dtype='int32')
    M = np.zeros(max_length, dtype='int32')
    V = np.zeros(max_length, dtype='int32')
    words = []
    while len(words) <= max_length and queue:
        word = queue.pop(0)
        if rnn_encode or (not word.is_punct and not word.is_space):
            words.append(word)
    words.sort()
    for j, token in enumerate(words):
        if token.text == '@placeholder':
            V[j] = 1
        elif token.text[:7] == '@entity':
            V[j] = int(re.search(r'\d+', token.text[7:]).group(0)) + 2
        if token.has_vector:
            X[j] = token.rank+1
            M[j] = 1
        else:
            X[j] = (token.shape % (nr_unk - 1)) + 2
        j += 1
        if j >= max_length:
            break
    return X, M, V

In [31]:
for i in range(focus.shape[0]):
    X, M, V = get_word_ids(nlp(focus['story'].iloc[i], parse=False, tag=False, entity=False), max_length=2000)
    print(i)
    print(X.max(), X.min())
    print(V.max(), V.min())

0
692207 0
496 0
1
341773 0
475 0
2
692207 0
498 0
3
692207 0
501 0
4
341773 0
487 0
5
525760 0
500 0
6
831419 0
490 0
7
956716 0
466 0
8
850601 0
480 0
9
850601 0
476 0
10
692207 0
468 0
11
453924 0
499 0
12
1158527 0
460 0
13
786991 0
490 0
14
453924 0
501 0
15
453924 0
495 0
16
453924 0
497 0
17
523582 0
487 0
18
453924 0
491 0
19
443588 0
492 0
20
218098 0
496 0
21
218098 0
494 0
22
956716 0
490 0
23
956716 0
501 0
24
218098 0
500 0
25
453924 0
494 0
26
218098 0
501 0
27
688727 0
499 0
28
443588 0
453 0
29
443588 0
445 0
30
971417 0
482 0
31
971417 0
489 0
32
971417 0
499 0
33
218098 0
490 0
34
971417 0
492 0
35
971417 0
496 0
36
971417 0
497 0
37
971417 0
500 0
38
682479 0
494 0
39
956716 0
494 0
40
218098 0
486 0
41
956716 0
500 0
42
383190 0
490 0
43
635482 0
493 0
44
688727 0
491 0
45
726813 0
494 0
46
805189 0
497 0
47
383190 0
497 0
48
805189 0
501 0
49
577985 0
477 0
50
577985 0
495 0
51
541318 0
496 0
52
805189 0
494 0
53
541318 0
495 0
54
523582 0
479 0
55
682479 0
459 0
5